### Import the libraries

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
notebook_path = Path(os.getcwd())
root_path = notebook_path.parent.absolute()
os.chdir(root_path)
str(root_path)

'/Users/Antoine/data_science_projects/natixis_challenge'

### Load the datasets

In [3]:
df_cockpit = pd.read_csv('./data/cockpit_20221221.csv').drop('Unnamed: 0', axis=1)
df_item_info = pd.read_csv('./data/item_info_20221221.csv').drop('Unnamed: 0', axis=1)
df_item_trend = pd.read_csv('./data/item_trend_20221221.csv').drop('Unnamed: 0', axis=1)
df_tmp_hosts_zabbix = pd.read_csv('./data/tmp_hosts_zabbix_20221221.csv').drop('Unnamed: 0', axis=1)
df_mycloud = pd.read_csv('./data/mycloud_20221221.csv').drop('Unnamed: 0', axis=1)

### Vizualise the datasets

In [4]:
print(df_cockpit.shape)
df_cockpit.head(2)

(5189, 22)


,name_department,iua,name_subfunction,name_function,name_server,model,name_state,name_environment,os,country,...,ram,number_core,number_cpu,type_cpu,electrical_power,server_parent,server_parent2,name_cluster,mycloud,bcloud
0,etu-bgc-etrading-architecture,AV9,Application Server,APPLICATION,SWDCFRAV9779,VIRTUAL_MACHINE,INFUNCTION,Dev,Windows 2016 Standard 10.0.14393,FRANCE,...,16777,2,2,INTEL(R) XEON(R) SILVER 4314 CPU @ 2.40GHZ,NaN,SEPCFRNXFX10808,NaN,NaN,Yes,NaN
1,etu-bgc-etrading-architecture,Y59,Application Server,APPLICATION,SLPAFRETR168,VIRTUAL_MACHINE,INFUNCTION,Prod,Linux 3.10.0-1160.76.1.el7.x86_64 RHEL7.9,FRANCE,...,7629,1,1,INTEL(R) XEON(R) CPU E5-2680 0 @ 2.70GHZ:X86_64,NaN,SEPIFRLINP029,NaN,NaN,No,NaN


In [5]:
print(df_item_info.shape)
df_item_info.head(2)

(10015, 3)


,itemid,hostid,item_type
0,264670,806,cpu
1,622332,4755,cpu


In [6]:
print(df_item_trend.shape)
df_item_trend.head(2)

(3866173, 6)


,itemid,clock,value_min,value_avg,value_max,item_type
0,264670,2021-05-27,5.789395,7.956363,50.884403,cpu
1,264670,2021-05-28,5.737689,10.279124,63.836303,cpu


In [7]:
print(df_mycloud.shape)
df_mycloud.head(5)

(35, 5)


,key,NameMyCloud,CPU,RAM,Price
0,1#2,small,1,2,332.15
1,1#4,small_2,1,4,445.30
2,1#6,small_3,1,6,547.50
3,1#8,small_4,1,8,667.95
4,1#12,small_6,1,12,1109.60


In [8]:
print(df_tmp_hosts_zabbix.shape)
df_tmp_hosts_zabbix.head(2)

(4365, 2)


,hostid,host
0,6,slpdfrora040a
1,800,SWPAFRWNETV44


### Calculate the price of the configuration
We only take into account the mycloud servors (we don't have the catalogue of the other servors).

In [10]:
print(f"{len(df_cockpit[df_cockpit.mycloud=='Yes'].ram.sort_values().unique())} different memory capacities of ram.")

80 different memory capacities of ram.


In [11]:
# Select mycloud servers
config = pd.DataFrame(df_cockpit[df_cockpit.mycloud=='Yes'][["iua", "ram", "number_cpu", "name_server"]].sort_values("ram"))
config["ram"] = config.ram/1000
config.head(2)

,iua,ram,number_cpu,name_server
342,GOS,1.76,1,SLDCFRNXGOS5320
694,Q63,1.76,1,SLUCFRNXQ630208


In [49]:
# Extract the list of servers (multiple applications correspond to one server)
info_servers = config[["ram", "number_cpu", "name_server"]].drop_duplicates()
print(f"{config.shape[0]-info_servers.shape[0]} servers host multiple applications.")
info_servers.head(3)

719 servers host multiple applications.


,ram,number_cpu,name_server
342,1.760,1,SLDCFRNXGOS5320
694,1.760,1,SLUCFRNXQ630208
2323,1.855,1,SLDCFRCDB481


In [50]:
def find_closest(ram, ram_refs):
    for index, ram_ref in enumerate(ram_refs):
        # Initialization
        if index==0:
            ram_final = ram_refs[0]

        # Recurrence
        else:
            if abs(ram-ram_ref) < abs(ram-ram_final):
                ram_final = ram_ref
    return ram_final
find_closest(12380, [1000, 5000, 10000, 15000, 20000])

10000

In [51]:
# Find the actual ram in the catalogue
info_servers["ram_ref"] = info_servers.ram.apply(lambda x: find_closest(x, df_mycloud.RAM))
info_servers.head(3)

,ram,number_cpu,name_server,ram_ref
342,1.760,1,SLDCFRNXGOS5320,2
694,1.760,1,SLUCFRNXQ630208,2
2323,1.855,1,SLDCFRCDB481,2


In [52]:
# Add catalogue info (price, server type, etc.)
info_servers = info_servers.merge(df_mycloud, how="left", left_on=["ram_ref", "number_cpu"], right_on=["RAM", "CPU"])
info_servers = info_servers.drop(["number_cpu", "ram_ref"], axis=1)
info_servers.head(3)

,ram,name_server,key,NameMyCloud,CPU,RAM,Price
0,1.760,SLDCFRNXGOS5320,1#2,small,1,2,332.15
1,1.760,SLUCFRNXQ630208,1#2,small,1,2,332.15
2,1.855,SLDCFRCDB481,1#2,small,1,2,332.15


In [53]:
# Calculate the price.
config_price = info_servers.Price.sum()
print(f"The total price of the config is {config_price}€.")

The total price of the config is 6502691.45€.


In [6]:
# Final function
def calculate_price(df_cockpit=df_cockpit, df_mycloud=df_mycloud):
    '''
    Calculate the price of a configuration.

    Args:
        df_cockpit : pandas.DataFrame = table of the configuration
        df_mycloud : pandas.DataFrame = table of the catalogue of prices of mycloud servors

    Return:
        price : float = total price of the configuration
    '''

    def find_closest(ram, ram_refs):
        '''
        Find the closest value to a float in a list of floats.

        Args:
            ram : float = value to find the closest of
            ram_refs : list(float) = reference values
            
        Return: 
            ram_final : float = closets reference value.
        '''
        for index, ram_ref in enumerate(ram_refs):
            # Initialization
            if index==0:
                ram_final = ram_refs[0]

            # Reccurence
            else:
                if abs(ram-ram_ref) < abs(ram-ram_final):
                    ram_final = ram_ref
        return ram_final

    # Extract usable data : mycloud servors only
    config = pd.DataFrame(df_cockpit[df_cockpit.mycloud=='Yes'][["iua", "ram", "number_cpu", "name_server"]].sort_values("ram"))
    # Convert in GBytes
    config["ram"] = config.ram/1000 

    # Extract the list of servers (multiple applications correspond to one server)
    info_servers = config[["ram", "number_cpu", "name_server"]].drop_duplicates()
    
    # Find the servors in the catalogue
    info_servers["ram_ref"] = info_servers.ram.apply(lambda x: find_closest(x, df_mycloud.RAM))

    # Add prices
    info_servers = info_servers.merge(df_mycloud, how="left", left_on=["ram_ref", "number_cpu"], right_on=["RAM", "CPU"])
    info_servers = info_servers.drop(["number_cpu", "ram_ref"], axis=1)

    # Calculate the price.
    config_price = info_servers.Price.sum()

    return config_price

In [45]:
f"Total price of the config: {calculate_price()}€."

'Total price of the config: 6502691.45€.'

Function added to src.utils.py.

### Estimate the cost of saturation

In [22]:
for type in df_item_trend.item_type.unique():
    print(f"{type} saturated {df_item_trend[(df_item_trend.item_type==type)*(df_item_trend.value_max==100)].shape[0]} times.")

cpu saturated 118949 times.
mem saturated 0 times.
citrix saturated 0 times.


If a server saturates, the pending executions queue up.  
  
Hypothesis : 
- there are only cpu saturation costs, no memory saturation cost
    
Two saturation states : 
- cpu_max=100 & cpu_avg!=100 <=>  saturation 50% of the time  &  30% of the user's time is wasted 
- cpu_max=100 & cpu_avg=100 <=>  saturation 100% of the time  &  60% of the user's time is wasted 
  
Parameters :
- salary (user's time waisted)
- value of the project (project delayed)

$$ saturationCost = saturationFactor*(0.6*numberUsers*salary + projectValue) $$
where saturationFactor can be either 1 or 0.5, depending on the type of the saturation (see hypothesis)


In [27]:
df2 = df_item_trend[df_item_trend.item_type.isin(["cpu", "mem"])].copy() # we remove citrix that is the number of session
df2['clock'] = pd.to_datetime(df2['clock'])

# Select a one year date range
end_date = df2.clock.max()
start_date = end_date.replace(year = end_date.year - 1)
df2 = df2[(df2['clock'] > start_date) & (df2['clock'] <= end_date)]
print(f"Dates going from {str(start_date)} to {str(end_date)}.")

print(df2.shape)
df2.head(3)

Dates going from 2021-12-20 00:00:00 to 2022-12-20 00:00:00.
(2647538, 6)


,itemid,clock,value_min,value_avg,value_max,item_type
208,264670,2021-12-21,4.176619,9.647727,65.765696,cpu
209,264670,2021-12-22,4.157618,9.746449,99.055944,cpu
210,264670,2021-12-23,4.130700,9.214908,57.616304,cpu


In [28]:
salary_year = 40000
number_user = 1
project_daily_value = 0

In [29]:
# Add a saturation cost column
# saturation_cost = saturation_factor*(number_user*salary_cost + project_daily_value)
salary_day = salary_year/365
saturation_cost_full_day = 0.6*number_user*salary_day + project_daily_value
df2["saturation_cost"] = 0
df2.loc[df2["value_avg"]==100, "saturation_cost"] = saturation_cost_full_day
df2.loc[(df2["value_avg"]!=100)&(df2["value_max"]==100), "saturation_cost"] = 0.5 * saturation_cost_full_day

df2.head(2)

,itemid,clock,value_min,value_avg,value_max,item_type,saturation_cost
208,264670,2021-12-21,4.176619,9.647727,65.765696,cpu,0.0
209,264670,2021-12-22,4.157618,9.746449,99.055944,cpu,0.0


In [32]:
# Calculate total saturation cost
total_saturation_cost = df2.saturation_cost.sum()
print(f"Total saturation cost : {round(total_saturation_cost, -3)}€.")

Total saturation cost : 2794000.0€.


In [13]:
def saturation_cost(df_item_trend,\
    salary_year=40000, 
    number_users=1, 
    project_daily_value=0, 
    saturation_threshold = 99,
    average_saturation_duration = 0.1,
    user_dependance = 0.6
    ):
    '''
    Estimate the total saturation cost over the df_item_trend period.

    Args:        
        df_item_trend : pd.DataFrame = table listing the cpu usage of all servers per day
        salary_year : int = annual average salary of the people working with the servers
        number_users : int = average number of users depending on the applications
        project_daily_value : daily financial impact of delay of the project 
        saturation_threshold : int = usage percentage from which we consider that the item is saturated
        average_saturation_duration : float = average duration of a saturation (proportion of one day)
        user_dependance : float = proportion of the user's time that depends on the app

    Output:
        total_saturation_cost : int = total saturation cost estimation in euros
    '''
    # Estimate the daily cost of saturation 
    # saturation_cost = saturation_factor*(number_user*salary_cost + project_daily_value)
    salary_day = salary_year/365
    saturation_cost_full_day = (user_dependance * number_users * salary_day) + project_daily_value

    # Add a saturation cost column
    df = df_item_trend.copy()
    df["saturation_cost"] = 0
    df.loc[df["value_avg"]>saturation_threshold, "saturation_cost"] = saturation_cost_full_day
    df.loc[(df["value_avg"]<saturation_threshold)&(df["value_max"]>saturation_threshold), "saturation_cost"]\
        = average_saturation_duration * saturation_cost_full_day

    # calculate total saturation cost
    total_saturation_cost = df.saturation_cost.sum()

    return total_saturation_cost

In [14]:
print(f"Total cost of saturation : {round(saturation_cost(df_item_trend), -3)}€.")

Total cost of saturation : 2069000.0€.


### Final cost of a configuration
We combine the configuration price and the cost of saturation. 
Let's calculate the average period covered by the time series, to know on which period we calculated the stauration price.

In [51]:
df_item_trend.head(2)

,itemid,clock,value_min,value_avg,value_max,item_type
0,264670,2021-05-27,5.789395,7.956363,50.884403,cpu
1,264670,2021-05-28,5.737689,10.279124,63.836303,cpu


In [16]:
def duration_avg(df_item_trend):
    '''
    Calculate the average duration covered by the time series of df_item_trend.

    Args:
        df_item_trend : pd.DataFrame([itemid, clock]) = table of the time series
    Output: 
        duration_avg : float = average duration
    '''
    df_item_trend["clock"] = pd.to_datetime(df_item_trend.clock).copy()
    df_avg_duration = df_item_trend[['itemid', "clock"]].rename(columns={'clock': 'max_date'}).groupby("itemid").max()
    df_avg_duration["min_date"] = df_item_trend[['itemid', "clock"]].groupby("itemid").min()
    df_avg_duration["duration_year"] = (df_avg_duration.max_date - df_avg_duration.min_date).apply(lambda x: x.total_seconds()/(365.25*24*60*60))
    
    duration_avg = df_avg_duration.duration_year.mean()

    return duration_avg

In [17]:
duration_mean = duration_avg(df_item_trend)
print(f"Average time-series duration in years : {duration_mean}.")

Average time-series duration in years : 1.2320902428594132.


We now combine the figures to get the annual cost of the configuration. 

In [19]:
saturation_cost(df_item_trend), calculate_price(df_cockpit, df_mycloud)

(9863835.616438357, 6502691.45)

In [18]:
total_config_price = saturation_cost(df_item_trend)/duration_mean + calculate_price(df_cockpit, df_mycloud)
total_config_price

NameError: name 'calculate_price' is not defined

### Test packaged functions

In [4]:
from src.pricing import find_closest, annual_config_price, total_saturation_cost, ts_duration_mean

In [10]:
def get_all_costs(df_cockpit,\
    df_mycloud, 
    df_item_trend,
    salary_year=40000, 
    number_users=1, 
    project_daily_value=0, 
    saturation_threshold=99):
    '''
    Main cost calculus fonction. Calculates both saturation cost and configuration price. 

    Args:
        df_cockpit : pandas.DataFrame(["RAM", "CPU", "Price"]) 
            = table of the configuration
        df_mycloud : pandas.DataFrame(["ram", "number_cpu", "name_server"]) 
            = table of the catalogue of prices of mycloud servors
        df_item_trend : pd.DataFrame([itemid, clock]) 
            = table of the time series
        salary_year : int 
            = annual average salary of the people working with the servers
        number_users : int 
            = average number of users depending on the applications
        project_daily_value : int 
            = daily financial impact of delay of the project 
        saturation_threshold : int 
            = usage percentage from which we consider that the item is saturated
    Outputs:
        annual_config_price : int = annual price of all the servers 
        total_saturation_cost : int = total cost of saturations over the period covered by df_item_trend 
        ts_average_duration : int = average duration covered by the time-series of df_item_trend
        total_annual_config_price : int = total annual price of the configuration (usage+saturation)
    '''

    annual_conf_price = annual_config_price(df_cockpit, df_mycloud)
    total_sat_cost = total_saturation_cost(df_item_trend,\
         salary_year, 
         number_users, 
         project_daily_value, 
         saturation_threshold)
    ts_average_duration = ts_duration_mean(df_item_trend)

    total_annual_config_price = total_sat_cost/ts_average_duration + annual_conf_price

    print(f"Annual configuration price : {int(annual_conf_price.round())}€.")
    print(f"Total saturation cost : {int(total_sat_cost.round())}€.")
    print(f"Time-series average duration : {ts_average_duration.round(3)} years.")
    print(f"Total annual configuration expenses (servers' usage and saturation) : {int(total_annual_config_price.round())}€.")

    return annual_conf_price, total_sat_cost, ts_average_duration, total_annual_config_price


In [7]:
ts_duration_mean(df_item_trend)

1.2320902428594132

In [6]:
2000000/1.23

1626016.2601626017

In [5]:
from src.pricing import get_all_costs
annual_conf_price, total_sat_cost, ts_average_duration, total_annual_config_price = get_all_costs(df_cockpit, df_mycloud, df_item_trend)

Annual configuration price : 6502691€.
Total saturation cost : 2069346€.
Time-series average duration : 1 years.
Total annual configuration expenses (servers' usage and saturation) : 8182232€.


In [11]:
annual_conf_price, total_sat_cost, ts_average_duration, total_annual_config_price = get_all_costs(df_cockpit, df_mycloud, df_item_trend)

Annual configuration price : 6502691€.
Total saturation cost : 9863836€.
Time-series average duration : 1.232 years.
Total annual configuration expenses (servers' usage and saturation) : 14508465€.


In [12]:
9863836/1.232

8006360.38961039